In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from src.template import template

/home/andrey/anaconda3/envs/aml_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model and tokenizer names
model_name ="google/gemma-7b-it"

# Load the model and tokenizer
device = torch.device("cuda:1")
model = AutoModelForCausalLM.from_pretrained(model_name).eval().to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]


In [3]:
model_params = {
    "max_new_tokens": 700,
}
message = [
    *template, 
    {
        "role": "user",
        "content": 
        """
          Course title: Advanced Machine Learning
          Short description: This course covers the following concepts: Advanced topics in machine learning; To develop research interest in the theory and application of machine learning.
        """,
    },
]
prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )
input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
generated_ids = model.generate(
    input_ids=input_ids.to(device),
    **model_params,
)
response = tokenizer.decode(generated_ids[0])

In [4]:
print(response[len(prompt):])

"Course Topics": [
            {
            "Section": "Advanced Supervised Learning",
            "Topics within the section": [
                "Deep Learning Fundamentals",
                "Convolutional Neural Networks",
                "Recurrent Neural Networks",
                "Deep Learning Architectures",
                "Transfer Learning",
                "Active Learning",
                "Bayesian Learning",
                "Deep Learning Applications"
            ]
            },
            {
            "Section": "Advanced Unsupervised Learning",
            "Topics within the section": [
                "Dimensionality Reduction",
                "Clustering Algorithms",
                "Association Rule Mining",
                "Graph Algorithms",
                "Deep Learning for Unsupervised Learning"
            ]
            },
            {
            "Section": "Advanced Topics in Machine Learning",
            "Topics within the section": [
               

In [5]:
s = '"Course Topics": '
import ast
res = {"Intended Learning Outcomes": ast.literal_eval(response[len(prompt)+len(s):][:-5])}
import json
with open('aml.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)